In [1]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

In [6]:
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    memory_key="chat_history",
    k = 10
)

history_counter = 0

def add_message(input, output):
    global history_counter
    history_counter += 1
    memory.save_context(
        {"input" : f"[{history_counter}] {input}"}, 
        {"output" : f"[{history_counter}] {output}"}
    )

def get_history():
    return memory.load_memory_variables({})

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너의 이름은 '살랑이'이야. 사용자가 반말을 요구하지 않는 이상 존댓말을 하도록 해. history_counter가 높은 것이 최근, 낮은 것이 비교적 오래전 대화 기록이야. [n]은 대화순서 표기용이니 실제로 대화할 때 이것이 드러나지 않도록 해."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt
)

In [7]:
print("※ 챗봇입니다. 종료하시려면 'exit'를 입력해주세요.")
while True:
    human_msg = input("You :")
    print(f"You : {human_msg}")
    
    if human_msg.lower() == "exit":
        break

    elif human_msg == "db":
        print(memory.load_memory_variables({}))

    else:
        print("AI : ", end="")
        ai_msg = chain.predict(question = human_msg)
        add_message(human_msg, ai_msg)
        print()

※ 챗봇입니다. 종료하시려면 'exit'를 입력해주세요.
You :  안녕
AI : 안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눠볼까요?
You : db
{'chat_history': [HumanMessage(content=' 안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눠볼까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='[1]  안녕', additional_kwargs={}, response_metadata={}), AIMessage(content='[1] 안녕하세요! 살랑이입니다. 오늘은 어떤 이야기 나눠볼까요?', additional_kwargs={}, response_metadata={})]}
You : 넌 누구야?
AI : 저는 살랑이라고 해요. 여러분과 대화하며 도움을 드리고 싶어요. 어떤 이야기를 나눠볼까요?
You : 난 누구지?
AI : 당신은 지금 저와 대화하고 계신 사용자님이세요. 혹시 더 궁금한 점이나 이야기하고 싶은 것이 있으시면 말씀해 주세요!
You : 내가 방금 뭐라고 했어?
AI : 당신은 방금 "난 누구지?"라고 말씀하셨어요. 혹시 더 궁금한 점이 있으시면 언제든 말씀해 주세요!
You : 그 전엔?
AI : 그 전에 하신 말씀은 "넌 누구야?"였어요. 계속해서 궁금한 점이나 이야기하고 싶은 것이 있으면 말씀해 주세요!
You : 그그 전엔?
AI : 그 전에 하신 말씀은 "안녕"이었어요. 언제든지 더 이야기하고 싶으신 게 있으면 말씀해 주세요!
You : exit
